In [ ]:
import numpy as np
import torch, torchvision

def load_dataset():
  train_set = torchvision.datasets.FashionMNIST("./data", download=True)
  test_set = torchvision.datasets.FashionMNIST("./data", download=True, train=False)

  X_train = train_set.data.numpy()
  labels_train = train_set.targets.numpy()
  X_test = test_set.data.numpy()
  labels_test = test_set.targets.numpy()

  X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2]))
  X_test = X_test.reshape((X_test.shape[0], X_test.shape[1]*X_test.shape[2]))
  X_train = X_train/255.0
  X_test = X_test/255.0

  return X_train, labels_train, X_test, labels_test

x_train, y_train, x_test, y_test = load_dataset()

100%|██████████| 26421880/26421880 [00:01<00:00, 13614278.40it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 201037.19it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3748679.98it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 22800714.88it/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd

LOGESTIC REGRESSION MODEL

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier_log = LogisticRegression ()
classifier_log.fit (x_train, y_train)
y_pred_log = classifier_log.predict (x_test)
cm_log = confusion_matrix (y_test, y_pred_log)
acc_log = accuracy_score (y_test, y_pred_log)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM MODEL

In [ ]:
from sklearn.svm import SVC
classifier_svm = SVC (kernel = 'rbf', random_state = 0)
classifier_svm.fit (x_train, y_train)
y_pred_svm = classifier_svm.predict (x_test)
cm_svm = confusion_matrix (y_test, y_pred_svm)
acc_svm = accuracy_score (y_test, y_pred_svm)

ACCURACY COMPARISON

In [ ]:
prediction_columns = ["NAME OF MODEL", "ACCURACY SCORE"]
df_pred = {"NAME OF MODEL" : ["LOGISTIC REGRESSION", "SVM"],
           "ACCURACY SCORE " : [acc_log, acc_svm]}
df_predictions = pd.DataFrame (df_pred)
df_predictions

,NAME OF MODEL,ACCURACY SCORE
0,LOGISTIC REGRESSION,0.8439
1,SVM,0.8828


HYPER-PARAMETER TUNING

In [ ]:
from sklearn.model_selection import GridSearchCV

LOGESTIC REGRESSION MODEL

In [ ]:
parameters = [{'penalty': ['l2'], 'C': [ 0.1, 1],
                'solver': ['newton-cg', 'liblinear']}]
grid_search = GridSearchCV(estimator = classifier_log,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = 1)
grid_result = grid_search.fit(x_train, y_train)
best_accuracy_log = grid_search.best_score_
best_parameters = grid_search.best_params_
best_model = grid_result.best_estimator_
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test acc:")
print(accuracy)
print("Val acc:")
print(best_accuracy_log)
print(best_parameters)

Test acc:
0.8461
Val acc:
0.8570666666666666
{'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}


SVM MODEL

In [ ]:
parameters = [{'C': [0.1, 1], 'kernel': ['linear', 'rbf'],
                'gamma': [0.1, 0.2]}]
grid_search = GridSearchCV(estimator = classifier_svm,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = 1)
grid_result = grid_search.fit(x_train, y_train)
best_accuracy_svm = grid_search.best_score_
best_parameters = grid_search.best_params_
best_parameters = grid_search.best_params_
best_model = grid_result.best_estimator_
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test acc:")
print(accuracy)
print("Val acc:")
print(best_accuracy_svm)
print(best_parameters)

FINAL ACCURACIES AFTER HYPER-PARAMETER TUNING

In [ ]:
import pandas as pd
prediction_columns = ["NAME OF MODEL", "ACCURACY SCORE", "BEST ACCURACY (AFTER HYPER-PARAMETER TUNING)"]
df_pred = {"NAME OF MODEL" : ["LOGISTIC REGRESSION", "SVM"],
           "ACCURACY SCORE " : [acc_log, acc_svm],
           "BEST ACCURACY (AFTER HYPER-PARAMETER TUNING)" : [best_accuracy_log, best_accuracy_svm]}
df_predictions = pd.DataFrame (df_pred)
df_predictions